In [1]:
from DiSuQ.Torch import models
from torch import tensor,stack
from numpy import arange,linspace,meshgrid,array,log,argsort,pi
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracyTarget,lossDegeneracyWeighted
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(32)

In [2]:
pairs = {'Lx':'Ly','Cx':'Cy','Jx':'Jy','CJx':'CJy'}

In [3]:
L_ = indE(1e-7); L0 = indE(5*1e-9); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(5*1e-16); C_ = capE(1e-14*5) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(5*1e-16/4); CJ_ = capE(1e-14/2) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 25. ; J_ = 5. ; print('Junction Bound(GHz):',J_,J0+J_)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 1.6346151294983178 34.32691771946467
Capacitance Bound(GHz): 0.38740458615415185 39.12786320156934
Shunt Bound(GHz): 3.8740458615415188 158.83588032320228
Junction Bound(GHz): 5.0 30.0


In [4]:
basis = [7,7,7] ; rep = 'Q'
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=10.,Ec=5.,El=10.,EcJ=50.,sparse=False,symmetry=True,_L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [5]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)},{'Lx':tensor(.45),'Ly':tensor(0.0)}]

In [6]:
def optimizationAnalysis(init,subspace,Search,success=1.):
    Loss,Success,Paths = [],[],[]
    for index,(init,(dLogs,dParams,dCircuit)) in enumerate(zip(init,Search)):
        Paths.append(dCircuit[subspace].to_numpy())
        loss = dLogs['loss'].to_numpy()
        Loss.append(loss[-1])
        if loss[-1] < success:
            Success.append(len(loss))
    return Paths,Loss,Success

In [7]:
def analysisPlotting(Optimization,Algo=['LBFGS']):
    paths = dict()
    losse = dict()
    for algo,(Paths,Loss,Success) in zip(Algo,Optimization):
        indices = argsort(Loss)[0]
        paths[algo] = Paths[indices]
        losse[algo] = Loss
    return paths,losse

In [8]:
def lossScapeBounds(paths):
    Paths = []
    for algo,path in paths.items():
        Paths.append(path)
    Paths = vstack(Paths)
    Paths = Paths[:, ~isnan(Paths).any(axis=0)]
    return Paths.min(0),Paths.max(0)

In [9]:
N = 10; subspace = ['Lx','Cx','Jx','CJx']
initials = truncNormalParameters(circuit,subspace,N=N,var=8)
len(initials)

10

In [10]:
N = 3; subspace = ['Lx','Cx','Jx','CJx']
initials = uniformParameters(circuit,subspace,N)
len(initials)

81

In [11]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

### Degeneracy domination

In [12]:
D0 = 1; delta0 = 0
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LBFGS = []
for index,parameter in enumerate(initials):
    print(optimizer.circuitState())
    print(index,parameter)
    optimizer.circuit.initialization(parameter)  
    optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
    LBFGS.append(optimizer.minimization(lossFunction,flux_profile,
                    method='L-BFGS-B',options=dict(ftol=1e-16,maxiter=15)))

{'Lx': 10.0, 'Ly': 10.0, 'Cx': 5.0, 'Cy': 5.0, 'Jx': 10.0, 'Jy': 10.0, 'CJx': 50.0, 'CJy': 50.0}
0 {'Lx': 8.173075647491588, 'Ly': 10.0, 'Cx': 9.685114653853796, 'Cy': 5.0, 'Jx': 6.25, 'Jy': 10.0, 'CJx': 38.740458615415186, 'CJy': 50.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.38786D-01    |proj g|=  1.11256D-01


In [ ]:
Result = [optimizationAnalysis(initials,subspace,LBFGS)]
paths_D,losse_D = analysisPlotting(Result)

In [ ]:
plotBox(losse_D,'Loss Distribution',export='pdf',size=(300,400))

### Delta domination

In [ ]:
D0 = 0; delta0 = 1
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LBFGS = []
for index,parameter in enumerate(initials):
    print(optimizer.circuitState())
    print(index,parameter)
    optimizer.circuit.initialization(parameter)  
    optimizer.parameters,optimizer.IDs = optimizer.circuitParameters()
    LBFGS.append(optimizer.minimization(lossFunction,flux_profile,
                    method='L-BFGS-B',options=dict(ftol=1e-16,maxiter=15)))

In [ ]:
Result = [optimizationAnalysis(initials,subspace,LBFGS)]
paths_delta,losse_delta = analysisPlotting(Result)

In [ ]:
plotBox(losse_delta,'Loss Distribution',export='pdf',size=(300,400))

### Comparison

In [ ]:
paths['optimal']  = [array([1.1*L_,1.1*C_,J0+.9*J_,CJ0+.9*CJ_])]

In [ ]:
paths.update(paths_delta)
paths.update(paths_D)

In [ ]:
Degeneracy,plots = dict(),dict()
for algo,path in paths.items():
    parameters = dict(zip(subspace,path[-1]))
    static.update(parameters)
    circuit.initialization(static)
    H_LC = circuit.chargeHamiltonianLC()
    H_J = circuit.josephsonCharge
    E0,(Ex1,Ex2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=True)
    Ex2 = Ex2.detach().numpy()
    Ex1 = Ex1.detach().numpy()
    degeneracy = log(Ex2/Ex1)
    Degeneracy[algo] = degeneracy
    plots[algo+'-E10'] = Ex1
    plots[algo+'-E20'] = Ex2

In [ ]:
plotCompare(flux_range[:,0],Degeneracy,'Degeneracy-Optimal Circuits','flux profile',export='pdf',size=(600,800))

In [ ]:
plotCompare(flux_range[:,0],plots,'Spectrum-Optimal Circuits','flux profile','energy(GHz)',export='pdf',size=(600,800))